In [ ]:
%run Common.ipynb

In [ ]:
import datetime, imageio, sys

import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image

In [ ]:
%matplotlib inline

### Custom Vision

In [ ]:
ENDPOINT = 'https://australiaeast.api.cognitive.microsoft.com/'

trainer = CustomVisionTrainingClient(custom_vision_training_key, endpoint=ENDPOINT)

projects = trainer.get_projects()

In [ ]:
for project in projects:
    print(project.id)
    print(project.name)

### Blob Storage

In [ ]:
containers = blob_service_list_containers(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key)

for container_name in containers:
    print(container_name)

### Create regions

In [ ]:
for container_name in containers:
    if container_name == 'site-season':
        blobs = blob_service_list_blobs(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, container_name)

        folders = set([blob.split('/')[0] for blob in blobs])

        for folder in folders:
            interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', container_name, folder)
            if not os.path.exists(interim_folder_path):
                os.makedirs(interim_folder_path)

        for blob_name in blobs:
            folder = blob_name.split('/')[0]
            file = blob_name.split('/')[1]
            
            filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(folder, file)
            
            entities = table_service_query_entities(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, healthy_habitat_ai_processed_table_name, filter)

            count = len([entity for entity in entities])

            if count == 0:
                print('Processing {0}'.format(blob_name))
                file_path = os.path.join(interim_folder_path, file)

                if not os.path.exists(file_path):
                    blob_service_get_blob_to_path(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, container_name, blob_name, file_path)

                start = datetime.datetime.now()
                
                image = imageio.imread(file_path)
                
                stop = datetime.datetime.now()
                print((stop - start).total_seconds())
                
                image = image[:,:,0:3]
                image_shape = image.shape
                print(image_shape)
                
                height = 228
                width = 304

                count = 0

                for y in range(0, image_shape[0], height):
                    for x in range(0, image_shape[1], width):
                        region = image[y:y + height, x:x + width]
                        
                        print('({0}, {1})({2}, {3})'.format(y, y + height, x, x + width))
                        
                        file_name = '{0}_Region_{1}.jpg'.format(file.split('.')[0], count)

                        buffer = io.BytesIO()
                        Image.fromarray(region).save(buffer, format='JPEG')

                        images = []

                        images.append(ImageFileCreateEntry(name=file_name, contents=buffer.getvalue()))

                        print('Uploading to Classification Project...')
                        result = trainer.create_images_from_files(custom_vision_para_grass_project_id, images=images)

                        if not result.is_batch_successful:
                            for image_create_result in result.images:
                                print("Image status: ", image_create_result.status)
                                
                        print('Uploading to Object Detection Project...')
                        result = trainer.create_images_from_files(custom_vision_magpie_geese_project_id, images=images)

                        if not result.is_batch_successful:
                            for image_create_result in result.images:
                                print("Image status: ", image_create_result.status)

                        count += 1

                entity = {'PartitionKey': folder, 'RowKey': file}
                table_service_insert(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, healthy_habitat_ai_processed_table_name, entity)
                
                os.remove(file_path)
                print(count)

In [ ]:
classification_tagged_image_count = trainer.get_tagged_image_count(custom_vision_para_grass_project_id)
print('Classification Tagged image count:', classification_tagged_image_count)

classification_untagged_image_count = trainer.get_untagged_image_count(custom_vision_para_grass_project_id)
print('Classification Untagged image count:', classification_untagged_image_count)

print('Classification Image count', classification_tagged_image_count + classification_untagged_image_count)

object_detection_tagged_image_count = trainer.get_tagged_image_count(custom_vision_magpie_geese_project_id)
print('Object Detection Tagged image count:', object_detection_tagged_image_count)

object_detection_untagged_image_count = trainer.get_untagged_image_count(custom_vision_magpie_geese_project_id)
print('Object Detection Untagged image count:', object_detection_untagged_image_count)

print('Object Detection Image count', object_detection_tagged_image_count + object_detection_untagged_image_count)